<div>
    <img src="https://storage.googleapis.com/kaggle-competitions/kaggle/22838/logos/header.png?t=2020-11-02-21-55-44" width="100%"/>
</div>

In [ ]:
from kaggle_environments import make, evaluate
from matplotlib import pyplot as plt
from difflib import SequenceMatcher
from tqdm import tqdm
import random
import numpy as np

<h3 align=center style="color:black; background:#FBE338; border:0">Definition</h3>

<div>
In computer science and operations research, a genetic algorithm (GA) is a metaheuristic inspired by the process of natural selection that belongs to the larger class of evolutionary algorithms (EA).<br> Genetic algorithms are commonly used to generate high-quality solutions to optimization and search problems by relying on biologically inspired operators such as mutation, crossover and selection.
</div>
    
Reference: [Wikipedia](https://en.wikipedia.org/wiki/Genetic_algorithm)

<h3 align=center style="color:black; background:#FBE338; border:0">Implementation</h3>

In [ ]:
%%writefile genetics.py
import random
from difflib import SequenceMatcher

class Genetics:
    '''
        Implementation of Genetics Algorithm
    '''
    class Individual:
        def __init__(self, string, fitness=0):
            self.string = string
            self.fitness = fitness

    def __init__(self, size, num_generations=150, population_size=900, mutation_rate=0.01):
        self.alphabet = "012"
        self.target = ''.join(random.choices(self.alphabet,k=size))
        self.num_generations = num_generations
        self.population_size = population_size
        self.str_len = len(self.target)
        self.mutation_rate = mutation_rate
      
    def generate_population(self):
        pop_fit = []
        pop = self.spawn_population(size=self.population_size, length=self.str_len)
        done = False
        for gen in range(self.num_generations):
            pop, avg_fit = self.evaluate_population(pop, self.target)
            pop_fit.append(avg_fit)
            new_pop = self.next_generation(pop, \
                size=self.population_size, length=self.str_len, mut_rate=self.mutation_rate)
            pop = new_pop
            for x in pop: 
                if x.string == self.target: 
                    done = True
            if done:
                break
        return pop
    
    def similar(self, a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    def spawn_population(self, length=26,size=100):
        pop = []
        for i in range(size):
            string = ''.join(random.choices(self.alphabet,k=length))
            individual = self.Individual(string)
            pop.append(individual)
        return pop
    
    def recombine(self, p1_, p2_):
        p1 = p1_.string
        p2 = p2_.string
        child1 = []
        child2 = []
        cross_pt = random.randint(0,len(p1))
        child1.extend(p1[0:cross_pt])
        child1.extend(p2[cross_pt:])
        child2.extend(p2[0:cross_pt])
        child2.extend(p1[cross_pt:])
        c1 = self.Individual(''.join(child1))
        c2 = self.Individual(''.join(child2))
        return c1, c2

    def mutate(self, x, mut_rate=0.01):
        new_x_ = []
        for char in x.string:
            if random.random() < mut_rate:
                new_x_.extend(random.choices(self.alphabet,k=1))
            else:
                new_x_.append(char)
        new_x = self.Individual(''.join(new_x_))
        return new_x
    
    def evaluate_population(self, pop, target):
        avg_fit = 0
        for i in range(len(pop)):
            fit = self.similar(pop[i].string, target)
            pop[i].fitness = fit
            avg_fit += fit
        avg_fit /= len(pop)
        return pop, avg_fit

    def next_generation(self, pop, size=100, length=26, mut_rate=0.3):
        new_pop = []
        while len(new_pop) < size:
            parents = random.choices(pop,k=2, weights=[x.fitness for x in pop])
            offspring_ = self.recombine(parents[0],parents[1])
            child1 = self.mutate(offspring_[0], mut_rate=mut_rate)
            child2 = self.mutate(offspring_[1], mut_rate=mut_rate)
            offspring = [child1, child2]
            new_pop.extend(offspring)
        return new_pop

In [ ]:
%%writefile genetics_choice.py

from genetics import Genetics
from random import randrange

def genetics_choice(observation, configuration):
    genetics = Genetics(size=1, num_generations=10, population_size=10)
    pop = genetics.generate_population()
    return int((pop[0].string)[0])

<h3 align=center style="color:black; background:#FBE338; border:0">Run two genetics algorithms against each other</h3>

In [ ]:
env = make(
    "rps", 
    configuration={
        "episodeSteps": 10
    }
)


In [ ]:
env.run(
    ["genetics_choice.py", "genetics_choice.py"]
)

env.render(mode="ipython", width=500, height=400)